In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,author,publicationTime,bodyText
0,0,2.209799e+09,10/02/2016 10:30,RT @350Australia: Adani Group's Aust #coal min...
1,1,2.209799e+09,10/02/2016 10:36,RT @avivaimhof: Poor old #coal. Now even #Viet...
2,2,4.306011e+07,10/02/2016 10:37,RT @market_forces: Funds have burned billions ...
3,3,2.209799e+09,10/02/2016 10:37,RT @avivaimhof: #Vietnam PM Announces Retreat ...
4,4,5.855606e+08,10/02/2016 10:41,RT @350Australia: Adani Group's Aust #coal min...


In [2]:
interval = input('Enter Interval in seconds : ')

Enter Interval in seconds : 300


In [3]:
df["publicationTime"] = pd.to_datetime(df["publicationTime"])
df['bodyText'].fillna('not found',inplace=True)


In [4]:
import time
import datetime
users = []
combine_data = []
counter  = 0
tweet_time = 0
old_timestamp = 0
row_count = 0
df_user = pd.DataFrame()
df_retweet = pd.DataFrame()
df_count = pd.DataFrame()
index_counter = 0
def filter_data(text, combine_data):
    for t in text.split(' '):
        if t.startswith('@'):
            combine_data.append(t)
    return combine_data

def filter_retweet(text, count):
    for index, t in enumerate(text.split(' ')):
        if index == 0 and t == 'RT':
            count += 1
    return count  
for i, row in df.iterrows():
    type = pd.isnull(row['publicationTime'])
    if not type:
        timestamp = time.mktime(row['publicationTime'].timetuple())
        if i == 0:
            old_timestamp = timestamp
        if tweet_time > old_timestamp + float(interval):
            index_counter += 1
            pub = datetime.datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
            temp1 = pd.DataFrame({'publicationTime': pub, 'users': len(set(combine_data))},index=[index_counter])
            temp2 = pd.DataFrame({'publicationTime': pub, 'retweet': counter},index=[index_counter])
            temp3 = pd.DataFrame({'publicationTime': pub, 'count': row_count},index=[index_counter])
            df_user = pd.concat([df_user, temp1])
            df_retweet = pd.concat([df_retweet, temp2])
            df_count = pd.concat([df_count, temp3])
            old_timestamp = timestamp
            combine_data = []
            counter = 0
            row_count = 0
        else:
            combine_data = filter_data(row['bodyText'], combine_data)
            counter = filter_retweet(row['bodyText'], counter)
            tweet_time = timestamp
            row_count += 1
print(df_count.head(10))
print(df_user.head(10)) 
print(df_retweet.head(10))

    count      publicationTime
1       2  2016-10-02 10:37:00
2       3  2016-10-02 10:48:00
3       2  2016-10-02 11:20:00
4       3  2016-10-02 11:46:00
5       5  2016-10-02 11:53:00
6       6  2016-10-02 12:03:00
7       2  2016-10-02 12:09:00
8       2  2016-10-02 12:19:00
9       2  2016-10-02 12:34:00
10      2  2016-10-02 12:47:00
        publicationTime  users
1   2016-10-02 10:37:00      2
2   2016-10-02 10:48:00      5
3   2016-10-02 11:20:00      3
4   2016-10-02 11:46:00      2
5   2016-10-02 11:53:00      1
6   2016-10-02 12:03:00      4
7   2016-10-02 12:09:00      2
8   2016-10-02 12:19:00      0
9   2016-10-02 12:34:00      1
10  2016-10-02 12:47:00      1
        publicationTime  retweet
1   2016-10-02 10:37:00        2
2   2016-10-02 10:48:00        3
3   2016-10-02 11:20:00        0
4   2016-10-02 11:46:00        2
5   2016-10-02 11:53:00        3
6   2016-10-02 12:03:00        4
7   2016-10-02 12:09:00        1
8   2016-10-02 12:19:00        0
9   2016-10-02 12:34:

In [5]:
#%matplotlib inline
#plt.bar(df_count['publicationTime'], df_count['count'], align='center')
#plt.title('Interval')
#plt.show()


In [6]:
def gen_window(df):
    window = []
    count = 1
    for i, row in df.iterrows():
        window.append("Window {}".format(count))
        count += 1
    return window

In [7]:
df_count['window'] = gen_window(df_count)
df_user['windows'] = gen_window(df_user)
df_retweet['windows'] = gen_window(df_retweet)
df_count.to_csv('count_with_interval.csv')
df_user.to_csv('count_with_users.csv')
df_retweet.to_csv('count_with_retweets.csv')